In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
import statsmodels.api as sm
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from linearmodels import PanelOLS, PooledOLS, IV2SLS

In [2]:
from  pydynpd import regression

In [3]:
df = pd.read_stata('blundell_bond_2000_production_function.dta')
df[["sales", "labor", "capital"]] = np.log(df[["sales", "labor", "capital"]])

In [4]:
df.head()

,id,year,sales,labor,capital
0,886.0,1982.0,4.579228,0.571544,3.577469
1,886.0,1983.0,4.472189,0.640801,3.610862
2,886.0,1984.0,4.567035,0.440832,3.694748
3,886.0,1985.0,4.885006,0.547543,3.796566
4,886.0,1986.0,4.999058,0.547543,3.976419


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4072 entries, 0 to 4071
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       4072 non-null   float32
 1   year     4072 non-null   float32
 2   sales    4072 non-null   float32
 3   labor    4072 non-null   float32
 4   capital  4072 non-null   float32
dtypes: float32(5)
memory usage: 111.3 KB


#### Q2.1. OLS with time dummies. Test the null hypothesis $\alpha_L + \alpha_K = 1$.

Suppose the production fucntion is $Y_{it} =  L_{it}^{\alpha_L} K_{it}^{\alpha_K} \exp(\omega_{it}+e_{it})$.
After taking log-transformation, we have the regression

\begin{align}
y_{it} =  \alpha_L l_{it} + \alpha_K k_{it} + \omega_{it}+e_{it}
\end{align}



In [6]:
dummies = pd.get_dummies(df['year']).rename(columns=lambda x: 'year_' + str(x))
df = pd.concat([df, dummies], axis=1)
df = df.set_index(['id', 'year'])

In [7]:
exog_ind = ['labor', 'capital'] + [col for col in df.columns if 'year' in col]

exog_var = df[exog_ind]
endog_var = df['sales']



model_OLS = sm.OLS(endog=endog_var, exog=exog_var).fit()

In [8]:
print(model_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                 1.425e+04
Date:                Mon, 14 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:51:34   Log-Likelihood:                -1527.7
No. Observations:                4072   AIC:                             3075.
Df Residuals:                    4062   BIC:                             3139.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
labor           0.5579      0.010     56.761      

Test $\alpha_L + \alpha_K = 1$ by F-test and t-test separately.

In [9]:
F_test = model_OLS.f_test('labor + capital = 1')
F_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[9.29374035]]), p=0.0023141886029365514, df_denom=4.06e+03, df_num=1>

In [10]:
R = np.zeros_like(model_OLS.params)
R[0:2] = [1, 1]
print(R)
q = 1

T_test = model_OLS.t_test((R, q))
T_test

[1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.9902      0.003     -3.049      0.002       0.984       0.996

#### Q2.2. Fixed Effects estimator with time dummies. Test $\eta_{i} = 0, \mbox{ } \forall i$.

Suppose $\omega_{it} = \gamma_{t} + \eta_{i}$. Then we can use time dummies to control $ \gamma_{t}$ and the fixed effect on entity to remove $\eta_{i}$.

In [11]:
model_FE = PanelOLS(dependent=endog_var, exog=exog_var, entity_effects=True).fit()

print(model_FE.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  sales   R-squared:                        0.7379
Estimator:                   PanelOLS   R-squared (Between):              0.9537
No. Observations:                4072   R-squared (Within):               0.7379
Date:                Mon, Aug 14 2023   R-squared (Overall):              0.9498
Time:                        13:51:35   Log-likelihood                    2302.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1111.5
Entities:                         509   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(9,3554)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):          6.321e+05
                            

In [12]:
formula = 'labor + capital = 1'

model_FE.wald_test(formula=formula)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 121.3166
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x10ab7bca640

According the above test, we can reject the null hypothesis $\alpha_L + \alpha_K = 1$ under the significant level $1\%$.
That is, the production fucntion is decreasing return-to-scale.

Also, for the test of the null hypothesis of $\eta_i = 0\mbox{ }, \forall i$, 
the F-test statistics is $38.901$ and the p-value is $0$. Thus, we can reject the null hypothesis of no time-invariant unobserved heterogeneity

We expect the FE estimator to be biased. The estimator may control for the endogeneity due to the fixed effect $\eta_i$ but not for the correlation between the regressors and transitory shock $e_{it}$.

#### Q2.3. Fixed Effects - Cochrane Orcutt estimator with time dummies. Test the two over-identifying restrictions of the model. 

Suppose $e_{it} = \rho e_{it-1} + u_{it}, \mbox{ } |\rho| < 1$ and $u_{it}$ is independent of other variables. We have
\begin{align}
y_{it} &=  \alpha_L l_{it} + \alpha_K k_{it} + \omega_{it}+e_{it}\\
\rho y_{it-1} &=  \rho\alpha_L l_{it-1} + \rho \alpha_K k_{it-1} + \rho \omega_{it-1}+\rho e_{it-1}.
\end{align}
Thus, the regression becomes
\begin{align}
y_{it} &= \rho y_{it-1} +  \alpha_L (l_{it}-\rho l_{it-1}) + \alpha_K (k_{it}-\rho  k_{it-1} ) + (\omega_{it}-\rho \omega_{it-1})+u_{it}\\
       &= \rho y_{it-1} +  \alpha_L l_{it} - \alpha_L \rho l_{it-1} + \alpha_K k_{it} - \alpha_K \rho  k_{it-1} + \eta^*_i + \gamma^*_t  +u_{it}
\end{align}

Here suppose $\omega_{it}-\rho \omega_{it-1} = \eta^*_i + \gamma^*_t$.

In [13]:
exog_var = pd.concat([df['sales'].groupby('id').shift(1), df['labor'], df['labor'].groupby('id').shift(1),
                      df['capital'], df['capital'].groupby('id').shift(1)], axis=1,
                     keys=['sales_L1', 'labor', 'labor_L1', 'capital', 'capital_L1'])

exog_var = pd.concat([exog_var.reset_index(), dummies], axis=1)
exog_var = exog_var.set_index(['id', 'year']).dropna()
exog_var = exog_var.loc[:,exog_var.columns != 'year_1982.0']

endog_var = df['sales'].groupby('id', group_keys=False).apply(lambda group: group.iloc[1:])



In [14]:
model_FE_CO = PanelOLS(dependent=endog_var, exog=exog_var, entity_effects=True).fit()

print(model_FE_CO.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  sales   R-squared:                        0.7825
Estimator:                   PanelOLS   R-squared (Between):              0.9745
No. Observations:                3563   R-squared (Within):               0.7825
Date:                Mon, Aug 14 2023   R-squared (Overall):              0.9717
Time:                        13:51:36   Log-likelihood                    2757.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      995.10
Entities:                         509   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                 F(11,3043)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):          6.641e+05
                            

In [15]:
b = model_FE_CO.params

c_b = b[0] + b[2] / b[1]
c_b_deriv = np.array([1, -b[2]/(b[1]**2), 1/b[1], 0, 0, 0, 0, 0, 0, 0, 0, 0])

testStat1 = c_b * (c_b_deriv @ (model_FE_CO.cov) @ np.transpose(c_b_deriv))**(-1) * c_b

testStat1

111.37450034582149

In [16]:
c_b = b[0] + b[4] / b[3]
c_b_deriv = np.array([1, 0, 0, -b[4]/(b[3]**2), 1/b[3], 0, 0, 0, 0, 0, 0, 0])

testStat2 = c_b * (c_b_deriv @ (model_FE_CO.cov) @ np.transpose(c_b_deriv))**(-1) * c_b

testStat2

21.13049776842449

In [17]:
c_b = np.array([b[0] + b[2] / b[1],  b[0] + b[4] / b[3]])
c_b_deriv = np.array([[1, -b[2]/(b[1]**2), 1/b[1], 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [1, 0, 0, -b[4]/(b[3]**2), 1/b[3], 0, 0, 0, 0, 0, 0, 0]])

testStat3 = np.transpose(c_b) @ np.linalg.inv(c_b_deriv @ (model_FE_CO.cov) @ np.transpose(c_b_deriv)) @ c_b

testStat3

112.42524086858583

#### Q2.4. Arellano-Bond GMM estimator with time dummies and non-serially correlated transitory shock.

Suppose 
\begin{align}
y_{it}&=  \alpha_L l_{it} + \alpha_K k_{it} + \omega_{it}+e_{it}\\
       \omega_{it} &=\eta_{i} + \delta_t
\end{align}
After taking the first difference, the equation becomes
\begin{align}
\Delta y_{it} &= \alpha_L \Delta l_{it} + \alpha_K \Delta k_{it} +  \Delta \omega_{it}+ \Delta e_{it}\\
                &= \alpha_L \Delta l_{it} + \alpha_K \Delta k_{it} +  \Delta \delta_{t}+ \Delta e_{it}\\
\end{align}

Since there is no serial correlated transitory shock in productivity, so $\{\Delta e_{it}\}$ is uncorrelated with $\{l_{it-j}, k_{it-j}, y_{it-j}\}_{j\geq 2}$.

Suppose $l_{it} = f_{l}(l_{it-1}, k_{it-1}, \omega_{it})$ and $k_{it} = f_{k}(l_{it-1}, k_{it-1}, \omega_{it})$.
Thus, we have the laggaed variables as the instruments for the regression. Let choose $\{l_{it-2}, k_{it-2}\}$ as valid instruments.


In [18]:
df = df.reset_index()

(a) According to the estimates, the coefficient of capital is very small and not significantly
different to zero. This is completely implausible.
(b) The test of serial correlation "Arellano-Bond test for AR(2) in first differences" tests
for the null hypothesis $E(\Delta u_{it} \Delta u_{it-2}) = 0$. The p-value of this test is $0.002$. The null
hypothesis is clearly rejected. This implies that $u_{it}$ is serially correlated, and therefore that
the instruments  are not valid.
In summary, the estimates do nor have economic sense and the restrictions of the model
are rejected.



#### Q2.5. Arellano-Bond GMM estimator with time dummies and AR(1) transitory shock.

Suppose $e_{it} = \rho e_{it-1} + u_{it}, \mbox{ } |\rho| < 1$ and $u_{it}$ is independent of other variables. We have
\begin{align}
y_{it} &=  \alpha_L l_{it} + \alpha_K k_{it} + \omega_{it}+e_{it}\\
\rho y_{it-1} &=  \rho\alpha_L l_{it-1} + \rho \alpha_K k_{it-1} + \rho \omega_{it-1}+\rho e_{it-1}.
\end{align}
Thus, the regression becomes
\begin{align}
y_{it} &= \rho y_{it-1} +  \alpha_L (l_{it}-\rho l_{it-1}) + \alpha_K (k_{it}-\rho  k_{it-1} ) + (\omega_{it}-\rho \omega_{it-1})+u_{it}\\
       &= \rho y_{it-1} +  \alpha_L l_{it} - \alpha_L \rho l_{it-1} + \alpha_K k_{it} - \alpha_K \rho  k_{it-1} + \eta^*_i + \gamma^*_t  +u_{it}
\end{align}

Here suppose $\omega_{it}-\rho \omega_{it-1} = \eta^*_i + \gamma^*_t$.

After taking the first difference, the equation becomes
\begin{align}
\Delta y_{it}  = \rho\Delta y_{it-1} +  \alpha_L \Delta l_{it} - \alpha_L \rho\Delta l_{it-1} + \alpha_K\Delta k_{it} - \alpha_K \rho\Delta  k_{it-1} + \Delta\gamma^*_t  +\Delta u_{it}
\end{align}

In [19]:
command_str='sales L1.sales labor L1.labor capital L1.capital  | gmm(sales, 2:.) gmm(labor, 2:.) gmm(capital, 2:.) | onestep nolevel timedumm'
mypyd = regression.abond(command_str, df, ['id', 'year'])


 Dynamic panel-data estimation, one-step difference GMM
 Group variable: id                               Number of obs = 3054    
 Time variable: year                              Min obs per group: 6    
 Number of instruments = 69                       Max obs per group: 6    
 Number of groups = 509                           Avg obs per group: 6.00 
+-------------+------------+---------------------+------------+-----------+-----+
|    sales    |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+-------------+------------+---------------------+------------+-----------+-----+
|   L1.sales  | 0.3264209  |      0.0521606      | 6.2579990  | 0.0000000 | *** |
|    labor    | 0.5128697  |      0.0892432      | 5.7468800  | 0.0000000 | *** |
|   L1.labor  | 0.0726061  |      0.0927269      | 0.7830099  | 0.4336213 |     |
|   capital   | 0.1318291  |      0.1179029      | 1.1181163  | 0.2635173 |     |
|  L1.capital | -0.2066298 |      0.0949589      | -2.1759916 | 0.0295

In summary, accounting for AR(1) transitory shocks has improved significantly the esti-
mates and the specification tests. However, the restrictions of the model are only marginally
"accepted" and, most importantly, the estimate for the coefficient of capital seems implau-
sibly small.


#### Q2.5. Blundell-Bond system estimator with time dummies and non-serially correlated transitory shock.

(a) According to the estimates, the technology is more intensive in labor than in capital.
But now both $L$ and $K$ are larger and the value of $L + K$ is very close CRS. In fact, the
test of CRS has a p-value of $0.7794$.
(b) The test of serial correlation "Arellano-Bond test for AR(2) in first differences" tests
for the null hypothesis $E(\Delta u_{it} \Delta u_{it-2}) = 0$. Similarly as for the Arellano-Bond estimator
without AR(1), the p-value of this test is practically zero. The null hypothesis is clearly
rejected. This implies that uit is serially correlated, and therefore neither AB nor BB instruments are valid.


#### Q2.6. Blundell-Bond system estimator with time dummies and AR(1) transitory shock.

In [31]:
command_str='sales L1.sales labor L1.labor capital L1.capital  | gmm(sales, 2:.) gmm(labor, 2:.) gmm(capital, 2:.) | onestep  timedumm'
mypyd = regression.abond(command_str, df, ['id', 'year'])

 Dynamic panel-data estimation, one-step system GMM
 Group variable: id                               Number of obs = 3054    
 Time variable: year                              Min obs per group: 6    
 Number of instruments = 88                       Max obs per group: 6    
 Number of groups = 509                           Avg obs per group: 6.00 
+-------------+------------+---------------------+------------+-----------+-----+
|    sales    |   coef.    | Corrected Std. Err. |     z      |   P>|z|   |     |
+-------------+------------+---------------------+------------+-----------+-----+
|   L1.sales  | 0.7035520  |      0.0409546      | 17.1788430 | 0.0000000 | *** |
|    labor    | 0.4484997  |      0.0929987      | 4.8226468  | 0.0000014 | *** |
|   L1.labor  | -0.2389360 |      0.0932566      | -2.5621363 | 0.0104030 |  *  |
|   capital   | 0.5333291  |      0.1094111      | 4.8745446  | 0.0000011 | *** |
|  L1.capital | -0.4439966 |      0.0953875      | -4.6546597 | 0.0000032 